# Détection d'inter-blocage (Plusieurs instances par type de ressources)

In [2]:
import numpy as np
import pandas as pd
import display

def append(df: pd.DataFrame, series: pd.Series) -> pd.DataFrame:
    return pd.concat((df, series.to_frame().T), ignore_index=True)

EXEMPLE = "001"

In [3]:
Disponible = pd.read_csv(f"./exemples/detection-interblocage/{EXEMPLE}/disponible.csv")
Requete = pd.read_csv(f"./exemples/detection-interblocage/{EXEMPLE}/requete.csv")
Allocation = pd.read_csv(f"./exemples/detection-interblocage/{EXEMPLE}/allocation.csv")

M = len(Disponible.columns)
N = len(Requete.index)

display.dataframes({'Allocation': Allocation, 'Requête': Requete, 'Disponible': Disponible})

,A,B,C
0,0,1,0
1,2,0,0
2,3,0,3
3,2,1,1
4,0,0,2
,A,B,C
0,0,0,0
1,2,0,2
2,0,0,0
3,1,0,0


1. Soit _Travail_ et _Fin_ deux tableaux de longueurs _m_ et _n_, respectivement
On initialise _Travail_ et _Fin_ comme suit:

In [4]:
Travail = Disponible.copy()

Fin = np.zeros((1, N), dtype=np.bool_)
Fin = pd.DataFrame(Fin)

for i in range(N):
   Fin.iloc[0] = (Allocation.iloc[i] == 0).all()

display.dataframes({'Travail': Travail, 'Fin': Fin})

Travail 
 
 
 
 A 
 B 
 C 
 
 
 
 
 0 
 0 
 0 
 0 
 
 
 Fin 
 
 
 
 0 
 1 
 2 
 3 
 4 
 
 
 
 
 0 
 False 
 False 
 False 
 False 
 False

2. Trouver un index _i_ tel que :

In [15]:
i = None

for j in range(N):
    if not Fin.iloc[-1, j] and (Requete.iloc[j] <= Travail.iloc[-1]).all():
        i = j
        break

print(f"i: {i}")

i: None


Si un tel _i_ n’existe pas passer à l’étape 4.

3. `Travail = Travail + Allocation[i]`  
   `Fin[i] = true`  
    Aller à étape 2

In [14]:
Travail = append(Travail, Travail.iloc[-1] + Allocation.iloc[i])
Fin = append(Fin, Fin.iloc[-1])

Fin.iloc[-1, i] = True

display.dataframes({'Travail': Travail, 'Fin': Fin})

Travail 
 
 
 
 A 
 B 
 C 
 
 
 
 
 0 
 0 
 0 
 0 
 
 
 1 
 0 
 1 
 0 
 
 
 2 
 3 
 1 
 3 
 
 
 3 
 5 
 1 
 3 
 
 
 4 
 7 
 2 
 4 
 
 
 5 
 7 
 2 
 6 
 
 
 Fin 
 
 
 
 0 
 1 
 2 
 3 
 4 
 
 
 
 
 0 
 False 
 False 
 False 
 False 
 False 
 
 
 1 
 True 
 False 
 False 
 False 
 False 
 
 
 2 
 True 
 False 
 True 
 False 
 False 
 
 
 3 
 True 
 True 
 True 
 False 
 False 
 
 
 4 
 True 
 True 
 True 
 True 
 False 
 
 
 5 
 True 
 True 
 True 
 True 
 True

In [16]:
Blocage = not Fin.iloc[-1].all()

print(f"Blocage = {Blocage}")

Blocage = False
